In [ ]:
# Import necessary libraries
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
import os
from pathlib import Path

# Import your modules
